In [1]:
from crimm.Modeller import TopologyLoader, ParameterLoader
from crimm.Modeller.SeqChainGenerator import SeqChainGenerator
from crimm.Visualization import View, show_nglview_multiple

We define a function to show licorice since the default rendering is "cartoon" and does not show the side chains.

In [23]:
def show_licorice(chain):
    view = View()
    view.load_entity(chain)
    view.clear_representations()
    view.add_representation('licorice')
    return view

def show_licorice_multiple(chains):
    view = View()
    for chain in chains:
        component = view.load_entity(chain)[0]
        view.clear_representations(component = component._index)
        view.add_representation('licorice', component = component._index)
    return view

## Generate a Chain Object from Sequence
The `SeqChainGenerator` class is used to construct RNA and polypeptide chain from sequences. The chain will be constructed from the *internal coordinate* (ic) definitions from **CHARMM36** residue topology files (rtf).

In [3]:
generator = SeqChainGenerator()

### Chain from One-letter Sequences for a Short RNA

The `chain_type` parameter is needed to identify the correct rtf. And of course, protein one-letter sequence is also accepted

In [4]:
# chain_type is case insensitive. RNA or polyribonucleotide can also be accepted
generator.set_sequence('CUCGAAUCA', chain_type = 'rna') 
rna_chain = generator.create_chain()
rna_chain

NGLWidget()

<Polyribonucleotide id=A Residues=9>


In [5]:
show_licorice(rna_chain)

View()

### Chain from Three-letter Sequece for ADA Tripeptide

The residue names in the three-letter sequences need to be separated by white spaces.

In [7]:
# Protein or Polypeptide(L) is also accepted
generator.set_three_letter_sequence('ALA ASP ALA', chain_type = 'polypeptide')
peptide_chain = generator.create_chain()
peptide_chain

NGLWidget()

<Polypeptide(L) id=A Residues=3>


In [8]:
show_licorice(peptide_chain)

View()

## Minimize the Structure

We can load the constructed chain into pyCHARMM by `load_chain` function in `crimm.Adaptors.pyCHARMMAdaptors`. Once it's loaded, all pyCHARMM functions can be called and applied to the loaded chain, just like it was loaded from pdb and psf files.

In [9]:
from crimm.Adaptors.pyCHARMMAdaptors import load_chain, load_parameters, load_topology
from crimm.Adaptors.pyCHARMMAdaptors import minimize, sync_coords, delete_atoms

## Generate the Topology and Apply Parameters from crimm

Before we can load the chain, the **topology** and **parameter** definitions need to be generated on the chain we just constructed. Currently, we still need to specify the `chain_type` for our chain, but it will become automated soon. The generation routine will be simplified very soon.

In [10]:
chain_type = 'protein'
topo = TopologyLoader(chain_type)
param = ParameterLoader(chain_type)
# We need to fill the ic table on the topology definitions, since 
# certain residue definitions will have empty tables to be filled by the parameter files
param.fill_ic(topo, preserve = True) # Preserve is TRUE by default

Here, we make a copy of the original chain for the structure comparison purposes

In [11]:
minimized_chain = peptide_chain.copy()

Currently, default terminal patches have not been implemented (since they need to be looked up from rtf files for different residues). We still need to specify patches when we generate the topology

In [12]:
topo.generate_chain_topology(minimized_chain, first_patch='ACE', last_patch='CT3')

Here, again we need to fill the ic table, since we just generated new residue definition with terminal patches, and these patch definitions' ic need to be filled from parameter files.

In [13]:
param.fill_ic(topo)

### Load Parameter and Topology Files
Before we can load the chain, the correct rtf and parameter files have to be loaded. We have utility function `load_topology` and `load_parameters` in the `pyCHARMMAdaptors` to help load the rtf and prm files correctly.

In [14]:
load_topology(topo)
load_parameters(param)

  
 CHARMM>     read rtf card -
 CHARMM>     name /tmp/tmpztue_g40
 VOPEN> Attempting to open::/tmp/tmpztue_g40::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *RTF LOADED FROM CRIMM
 TITLE> 36  2
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /tmp/tmp7ppigpea -
 CHARMM>     flex
 VOPEN> Attempting to open::/tmp/tmp7ppigpea::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *PRM LOADED FROM CRIMM
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> JAN. 2016 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 PARMIO> NONBOND, HBOND lists and IMAGE atoms cleared.
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


A wrapper function to delete all atoms in pyCHARMM. If there is not atom exists, an error will be raised.

In [64]:
delete_atoms()


 Message from MAPIC: Atom numbers are changed.


True


 Message from MAPIC:          6 residues deleted.

 Message from MAPIC:          1 segments deleted.
 DELTIC:        82 bonds deleted
 DELTIC:       142 angles deleted
 DELTIC:       195 dihedrals deleted
 DELTIC:        25 improper dihedrals deleted
 DELTIC:         6 crossterm maps deleted
 DELTIC:        10 donors deleted
 DELTIC:        11 acceptors deleted


In [15]:
load_chain(minimized_chain)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmpxuxezq9t
 VOPEN> Attempting to open::/tmp/tmpxuxezq9t::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     3 RESIDUES
          ALA ASP ALA 
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS  1 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  0  HSE -  0  HSP -  0  ASP -  1  GLU -  0  LYS -  0  TYR -  0
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 THE PATCH 'ACE' WILL BE USED FOR THE FIRST RESIDUE
 THE PATCH 'CT3' WILL BE USED FOR THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its identifier is PROA.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =        3
         Number of atoms        

There is a wrapper function for steepest descend and adopted basis Newton-Raphson minimization method.

In [16]:
minimize(sd_nstep=200, abnr_nstep=0)

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:      3
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSHIft   VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 14.000 CTEXNB =999.000 CTONNB = 10.000 CTOFNB = 12.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 <MAKINB> with mode   5 found    118 exclusions and     97 interactions(1-4)
 <MAKGRP> found     27 group exclusio

## Synchronize the Structure with pyCHARMM
Since the coordinates are minimized in pyCHARMM, we need to update the coords in our chain object by calling `sync_coords` function. This need to be called whenenver a computation is done and you want to update the structure on crimm side. 

**Note**: if there is any structure descripency between the pyCHARMM and crimm chain, the synchronization will be aborted. This is likely due to modification of the crimm chain after it was loaded, or a wrong chain is supplied as the function argument.

In [17]:
sync_coords(minimized_chain)

Synchronized: <Polypeptide(L) id=A Residues=3>


In [18]:
minimized_chain

NGLWidget()

<Polypeptide(L) id=A Residues=3>


### Compare the Minimized Structure and the Original

In [24]:
show_licorice_multiple([minimized_chain, peptide_chain])

View()